In [7]:
import sqlite3
import pandas

In [8]:
pandas.set_option('display.max_rows', 500)

In [9]:
sqlite_file = 'lahman2014.sqlite'
conn = sqlite3.connect(sqlite_file)

## Part 1: Wrangling
### Problem 1

In [10]:
stats_query = "SELECT teamID, yearID, franchID, W, G, (W * 1.0) / (G * 1.0) * 100 as win_percentage FROM Teams"
team_stats = pandas.read_sql(stats_query, conn)
team_stats

,teamID,yearID,franchID,W,G,win_percentage
0,BS1,1871,BNA,20,31,64.516129
1,CH1,1871,CNA,19,28,67.857143
2,CL1,1871,CFC,10,29,34.482759
3,FW1,1871,KEK,7,19,36.842105
4,NY2,1871,NNA,16,33,48.484848
...,...,...,...,...,...,...
2770,PIT,2014,PIT,88,162,54.320988
2771,SDN,2014,SDP,77,162,47.530864
2772,SFN,2014,SFG,88,162,54.320988
2773,SLN,2014,STL,90,162,55.555556


In [11]:
salary_query = "SELECT yearID, teamID, salary  FROM Salaries"

team_salaries = pandas.read_sql(salary_query, conn)
team_salaries

,yearID,teamID,salary
0,1985,ATL,870000.0
1,1985,ATL,550000.0
2,1985,ATL,545000.0
3,1985,ATL,633333.0
4,1985,ATL,625000.0
...,...,...,...
24753,2014,WAS,3450000.0
24754,2014,WAS,3975000.0
24755,2014,WAS,20000000.0
24756,2014,WAS,7500000.0


In [19]:
relation_query = '''WITH total_payroll as 
                    (SELECT teamID, 
                            yearID, 
                            sum(salary) as payroll 
                    FROM Salaries GROUP BY teamID, yearID)
                    SELECT Teams.teamID,
                        Teams.yearID,
                        Teams.lgID,
                        payroll,
                        franchID,
                        rank, 
                        W,
                        G, 
                        ((W * 1.0 / G * 1.0) * 100) as win_percentage
                    FROM total_payroll, Teams
                    WHERE total_payroll.yearID = Teams.yearID and
                    total_payroll.teamID = Teams.teamID'''
relation = pandas.read_sql(relation_query, conn)
relation

,teamID,yearID,lgID,payroll,franchID,Rank,W,G,win_percentage
0,ATL,1985,NL,14807000.0,ATL,5,66,162,40.740741
1,BAL,1985,AL,11560712.0,BAL,4,83,161,51.552795
2,BOS,1985,AL,10897560.0,BOS,5,81,163,49.693252
3,CAL,1985,AL,14427894.0,ANA,2,90,162,55.555556
4,CHA,1985,AL,9846178.0,CHW,3,85,163,52.147239
...,...,...,...,...,...,...,...,...,...
853,SLN,2014,NL,120693000.0,STL,1,90,162,55.555556
854,TBA,2014,AL,72689100.0,TBD,4,77,162,47.530864
855,TEX,2014,AL,112255059.0,TEX,5,67,162,41.358025
856,TOR,2014,AL,109920100.0,TOR,3,83,162,51.234568


In [20]:
relation

,teamID,yearID,lgID,payroll,franchID,Rank,W,G,win_percentage
0,ATL,1985,NL,14807000.0,ATL,5,66,162,40.740741
1,BAL,1985,AL,11560712.0,BAL,4,83,161,51.552795
2,BOS,1985,AL,10897560.0,BOS,5,81,163,49.693252
3,CAL,1985,AL,14427894.0,ANA,2,90,162,55.555556
4,CHA,1985,AL,9846178.0,CHW,3,85,163,52.147239
...,...,...,...,...,...,...,...,...,...
853,SLN,2014,NL,120693000.0,STL,1,90,162,55.555556
854,TBA,2014,AL,72689100.0,TBD,4,77,162,47.530864
855,TEX,2014,AL,112255059.0,TEX,5,67,162,41.358025
856,TOR,2014,AL,109920100.0,TOR,3,83,162,51.234568


In [26]:
team_query = "SELECT teamID FROM Salaries WHERE teamID = WAS"

one_team = pandas.read_sql(team_query, conn)
one_team

DatabaseError: Execution failed on sql 'SELECT teamID FROM Salaries WHERE teamID = WAS': no such column: WAS

In [ ]:
relationdos_query = '''WITH total_payroll as 
                    (SELECT teamID, 
                            yearID, 
                            sum(salary) as payroll 
                    FROM Salaries 
                    GROUP BY teamID, yearID)
                    INNER JOIN
                    SELECT Teams.teamID,
                        Teams.yearID,
                        Teams.lgID,
                        payroll,
                        franchID,
                        rank, 
                        W,
                        G, 
                        ((W * 1.0 / G * 1.0) * 100) as win_percentage
                    FROM total_payroll, Teams
                    WHERE total_payroll.yearID = Teams.yearID and
                    total_payroll.teamID = Teams.teamID'''
relationdos = pandas.read_sql(relationdos_query, conn)
relationdos

#### Missing data: 
Teams starts from the year 1871, whereas Salaries starts from 1985. Used WITH at the beginning of the query instead of a join, which the prompt asks for so we might have to change that. 

## Part 2: Exploratory Data Analysis
### Payroll Distribution, Problem 2